# Preprocesamiento Mejorado - Pump it Up Dataset

Este notebook implementa un preprocesamiento mejorado que incluye:
1. **Procesamiento de variables temporales** (extracción de características de fechas)
2. **KNN Imputer** para imputación de valores faltantes
3. **Eliminación de variables muy correlacionadas** (umbral > 0.95) con explicación detallada de decisiones

Dataset: Pump it Up: Data Mining the Water Table
Fuente: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## 1. Carga de datos

In [ ]:
# Cargar datos de entrenamiento y prueba
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_test: {y_test.shape}")

## 2. Exploración inicial de datos

In [ ]:
# Información general del dataset
X_train.info()

In [ ]:
# Valores faltantes
print("Valores faltantes por columna:")
missing_values = X_train.isna().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])

print("\nPorcentaje de valores faltantes:")
missing_pct = (X_train.isna().mean() * 100).sort_values(ascending=False)
print(missing_pct[missing_pct > 0])

## 3. Preprocesamiento mejorado

### 3.1 Eliminar variables que no aportan información

In [ ]:
# Eliminar variables que no se usan (aplicar a train y test)
variables_a_eliminar_inicial = ["scheme_name","scheme_management", "id","date_recorded","recorded_by","num_private "]
#NO APORTAN INFORMACIÓN ÚTIL 

X_train = X_train.drop(columns=variables_a_eliminar_inicial, errors="ignore")
X_test = X_test.drop(columns=variables_a_eliminar_inicial, errors="ignore")

print(f"Variables después de eliminar: X_train={X_train.shape[1]}, X_test={X_test.shape[1]}")

### 3.2 Separar variables categóricas y numéricas

In [ ]:
# Separar variables categóricas y numéricas
columnas_categoricas = X_train.select_dtypes(include="object").columns.tolist()
columnas_numericas = X_train.select_dtypes(exclude="object").columns.tolist()

print(f"Variables categóricas: {len(columnas_categoricas)}")
print(f"Variables numéricas: {len(columnas_numericas)}")
print(f"\nCategóricas: {columnas_categoricas}")
print(f"\nNuméricas: {columnas_numericas}")

# Identificar variables temporales
variables_temporales = []
if "date_recorded" in X_train.columns:
    variables_temporales.append("date_recorded")
if "construction_year" in X_train.columns:
    variables_temporales.append("construction_year")

if variables_temporales:
    print(f"\nVariables temporales identificadas: {variables_temporales}")
else:
    print("\nNo se encontraron variables temporales")

### 3.4 Convertir ceros problemáticos a NaN

In [ ]:
# Identificar columnas donde el 0 puede ser un valor faltante
zero_as_nan_cols = [
    "amount_tsh",
    "gps_height",
    "longitude",
    "population",
    "num_private",
    "construction_year"
]

# Convertir ceros a NaN en estas columnas (aplicar a train y test)
X_train_num = X_train[columnas_numericas].copy()
X_test_num = X_test[columnas_numericas].copy()

X_train_num[zero_as_nan_cols] = X_train_num[zero_as_nan_cols].replace(0, np.nan)
X_test_num[zero_as_nan_cols] = X_test_num[zero_as_nan_cols].replace(0, np.nan)

print(f"Valores NaN en X_train después de convertir ceros:")
print(X_train_num.isna().sum()[X_train_num.isna().sum() > 0])
print(f"\nValores NaN en X_test después de convertir ceros:")
print(X_test_num.isna().sum()[X_test_num.isna().sum() > 0])

### 3.5 Imputación con KNN Imputer

In [ ]:
# Preparar datasets para KNN Imputer
# Convertir todas las variables categóricas a numéricas usando pd.factorize()
# IMPORTANTE: Usar los mismos códigos de factorize para train y test
X_train_for_knn = X_train.copy()
X_test_for_knn = X_test.copy()

print("Convirtiendo atributos categóricos a numéricos con pd.factorize()...")
# Guardar los mapeos de factorize para aplicar los mismos códigos a test
factorize_mappings = {}

for column in columnas_categoricas:
    if column in X_train_for_knn.columns:
        # Aplicar factorize en train
        codes, uniques = pd.factorize(X_train_for_knn[column])
        X_train_for_knn[column] = codes
        X_train_for_knn[column].replace(-1, np.nan, inplace=True)
        
        # Guardar el mapeo para aplicar a test
        factorize_mappings[column] = uniques
        
        # Aplicar el mismo mapeo a test
        if column in X_test_for_knn.columns:
            # Mapear valores de test usando los códigos de train
            X_test_for_knn[column] = X_test_for_knn[column].map(dict(zip(uniques, range(len(uniques)))))
            # Valores no vistos en train se convierten en NaN
            X_test_for_knn[column] = X_test_for_knn[column].where(X_test_for_knn[column].notna(), np.nan)

print(f"✓ Variables categóricas convertidas a numéricas")
print(f"Forma X_train: {X_train_for_knn.shape}, Valores faltantes: {X_train_for_knn.isna().sum().sum()}")
print(f"Forma X_test: {X_test_for_knn.shape}, Valores faltantes: {X_test_for_knn.isna().sum().sum()}")

In [ ]:
# Aplicar KNN Imputer: fit en train, transform en train y test
print("Aplicando KNN Imputer (esto puede tardar unos minutos)...")
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes probar con otro valor de vecinos

# Ajustar el imputador con train y transformar train y test
X_train_imputed = pd.DataFrame(
    knn_imputer.fit_transform(X_train_for_knn),
    columns=X_train_for_knn.columns,
    index=X_train_for_knn.index
)

X_test_imputed = pd.DataFrame(
    knn_imputer.transform(X_test_for_knn),
    columns=X_test_for_knn.columns,
    index=X_test_for_knn.index
)

print("✓ KNN Imputer completado")
print(f"X_train - Valores faltantes después de imputación: {X_train_imputed.isna().sum().sum()}")
print(f"X_test - Valores faltantes después de imputación: {X_test_imputed.isna().sum().sum()}")
print(f"\nVerificación X_train:")
missing_after_train = X_train_imputed.isnull().sum()
print(missing_after_train[missing_after_train > 0] if missing_after_train.sum() > 0 else "✓ No hay valores faltantes")
print(f"\nVerificación X_test:")
missing_after_test = X_test_imputed.isnull().sum()
print(missing_after_test[missing_after_test > 0] if missing_after_test.sum() > 0 else "✓ No hay valores faltantes")

In [ ]:
# Los datasets ya están completamente imputados y todas las variables son numéricas
# Separar en numéricas originales y categóricas (ahora convertidas a numéricas)
X_train_num_imp = X_train_imputed[columnas_numericas].copy()
X_test_num_imp = X_test_imputed[columnas_numericas].copy()

# Las categóricas ahora están como números (después de factorize y KNN)
# Podemos mantenerlas así o volver a convertirlas a categóricas si es necesario
X_train_cat_imp = X_train_imputed[columnas_categoricas].copy()
X_test_cat_imp = X_test_imputed[columnas_categoricas].copy()

print("✓ Datasets imputados listos")
print(f"X_train - Variables numéricas: {X_train_num_imp.shape[1]}, categóricas: {X_train_cat_imp.shape[1]}")
print(f"X_test - Variables numéricas: {X_test_num_imp.shape[1]}, categóricas: {X_test_cat_imp.shape[1]}")

In [ ]:
# Los datasets ya están completamente imputados y todas las variables son numéricas
X_train_imputado = X_train_imputed.copy()
X_test_imputado = X_test_imputed.copy()

print(f"X_train completo imputado: {X_train_imputado.shape}")
print(f"X_test completo imputado: {X_test_imputado.shape}")
print(f"X_train - Valores faltantes: {X_train_imputado.isna().sum().sum()}")
print(f"X_test - Valores faltantes: {X_test_imputado.isna().sum().sum()}")
print(f"\n✓ Todas las variables están ahora como numéricas (categóricas convertidas con factorize)")

In [ ]:
# Los datasets ya están completos y listos para usar
# Todas las variables (numéricas y categóricas) están ahora como numéricas
print(f"\n✓ Preprocesamiento con KNN Imputer completado")
print(f"X_train final: {X_train_imputado.shape}")
print(f"X_test final: {X_test_imputado.shape}")
print(f"Todas las variables son numéricas (categóricas convertidas con pd.factorize())")

## 4. Eliminación de variables muy correlacionadas

In [ ]:
# Crear una copia para calcular correlaciones
# Las variables categóricas ya están como numéricas (después de factorize y KNN)
# así que podemos calcular correlaciones directamente
X_train_for_corr = X_train_imputado.copy()

print("✓ Dataset listo para análisis de correlación")
print("  (Las categóricas ya están como numéricas después de factorize)")

In [ ]:
# Calcular matriz de correlación
correlation_matrix = X_train_for_corr.corr().abs()

# Crear máscara para la parte superior de la matriz (para evitar duplicados)
upper_triangle = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)

# Encontrar pares de variables con correlación muy alta (umbral: 0.95)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if upper_triangle[i, j] and correlation_matrix.iloc[i, j] > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

print(f"Variables muy correlacionadas (correlación > 0.95): {len(high_corr_pairs)} pares")
if high_corr_pairs:
    print("\nPrimeros 10 pares encontrados:")
    for var1, var2, corr in high_corr_pairs[:10]:
        print(f"  {var1} <-> {var2}: {corr:.4f}")

In [ ]:
# Eliminar variables específicas que no aportan información útil (aplicar a train y test)
variables_a_eliminar = [
    "id",
    "water_quality",
    "payment",
    "source",
    "source_class",
    "extraction_type_group",
    "quantity_group",
    "waterpoint_type_group",
    "management",
    "region_code",
    "district_code"
]

# Verificar qué variables existen en el dataset antes de eliminar
variables_existentes = [var for var in variables_a_eliminar if var in X_train_imputado.columns]
variables_no_existentes = [var for var in variables_a_eliminar if var not in X_train_imputado.columns]

print("Eliminando variables específicas...")
print(f"\nVariables a eliminar que existen en el dataset ({len(variables_existentes)}):")
for var in variables_existentes:
    print(f"  - {var}")

if variables_no_existentes:
    print(f"\nVariables a eliminar que NO existen en el dataset ({len(variables_no_existentes)}):")
    for var in variables_no_existentes:
        print(f"  - {var}")

# Eliminar las variables de ambos datasets
X_train_imputado = X_train_imputado.drop(columns=variables_existentes, errors="ignore")
X_test_imputado = X_test_imputado.drop(columns=variables_existentes, errors="ignore")

# Actualizar las listas de columnas
columnas_categoricas = [c for c in columnas_categoricas if c not in variables_existentes]
columnas_numericas = [c for c in columnas_numericas if c not in variables_existentes]

print(f"\n✓ Variables eliminadas: {len(variables_existentes)}")
print(f"✓ Variables restantes en X_train_imputado: {X_train_imputado.shape[1]}")
print(f"✓ Variables restantes en X_test_imputado: {X_test_imputado.shape[1]}")

## 5. Preparación final para modelos

In [ ]:
# Después de eliminar variables correlacionadas, el dataset ya está listo
# Todas las variables (numéricas y categóricas) están como numéricas
print(f"Variables numéricas originales: {len([c for c in columnas_numericas if c in X_train_imputado.columns])}")
print(f"Variables categóricas (convertidas a numéricas): {len([c for c in columnas_categoricas if c in X_train_imputado.columns])}")
print(f"Total de variables: {X_train_imputado.shape[1]}")

In [ ]:
# Las variables categóricas ya están como numéricas (después de factorize y KNN)
# No necesitamos codificarlas de nuevo, ya están listas para los modelos
print("✓ Todas las variables están listas para los modelos")
print("  (Las categóricas ya están como numéricas después de factorize y KNN)")

In [ ]:
# Los datasets ya están completos y listos para los modelos
X_train_prep = X_train_imputado.copy()
X_test_prep = X_test_imputado.copy()

print("\n" + "="*50)
print("RESUMEN DEL PREPROCESAMIENTO")
print("="*50)
print(f"X_train - Forma final: {X_train_prep.shape}")
print(f"X_test - Forma final: {X_test_prep.shape}")
print(f"X_train - Valores faltantes: {X_train_prep.isna().sum().sum()}")
print(f"X_test - Valores faltantes: {X_test_prep.isna().sum().sum()}")
print(f"Variables numéricas originales: {len([c for c in columnas_numericas if c in X_train_prep.columns])}")
print(f"Variables categóricas (convertidas a numéricas): {len([c for c in columnas_categoricas if c in X_train_prep.columns])}")
print(f"\n✓ Datasets listos para entrenar modelos")
print(f"  (Todas las variables son numéricas)")

## 6. Verificación final

In [ ]:
# Verificar que no hay valores faltantes
assert X_train_prep.isna().sum().sum() == 0, "¡X_train aún tiene valores faltantes!"
assert X_test_prep.isna().sum().sum() == 0, "¡X_test aún tiene valores faltantes!"
print("✓ Verificación: No hay valores faltantes en train ni test")

# Verificar que las formas coinciden
assert X_train_prep.shape[0] == X_train.shape[0], "¡El número de filas de X_train no coincide!"
assert X_test_prep.shape[0] == X_test.shape[0], "¡El número de filas de X_test no coincide!"
assert X_train_prep.shape[1] == X_test_prep.shape[1], "¡El número de columnas no coincide entre train y test!"
print(f"✓ Verificación: Número de filas correcto (X_train: {X_train_prep.shape[0]}, X_test: {X_test_prep.shape[0]})")
print(f"✓ Verificación: Número de columnas coincide ({X_train_prep.shape[1]})")

print("\n✓ Preprocesamiento completado exitosamente")

## 7. Guardar datos preprocesados

In [ ]:
# Guardar los datos preprocesados en archivos CSV para usar en los modelos
X_train_prep = X_train_imputado.copy()
X_test_prep = X_test_imputado.copy()

X_train_prep.to_csv('X_train_prep.csv', index=False)
X_test_prep.to_csv('X_test_prep.csv', index=False)

print("✓ Datos preprocesados guardados:")
print(f"  - X_train_prep.csv: {X_train_prep.shape}")
print(f"  - X_test_prep.csv: {X_test_prep.shape}")
print(f"\n✓ Datasets listos para entrenar modelos")